In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
import pickle
import datacompy
import os, sys
import numpy as np
import re
import ast
import gower
from pathlib import Path

# narzedzia
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    cross_val_predict,
    learning_curve,
    RepeatedStratifiedKFold,
    GridSearchCV,
    RandomizedSearchCV
)
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix, silhouette_score, accuracy_score, pairwise_distances, make_scorer, precision_score, f1_score, roc_auc_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import tree
from sklearn.tree import plot_tree
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from scipy import stats
from scipy.stats import chi2_contingency
from scipy.spatial import distance
from joblib import dump, load

# modele
from xgboost import XGBClassifier, XGBRFClassifier
#from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# methods
from imblearn.under_sampling import ClusterCentroids, NearMiss
from scipy.optimize import differential_evolution
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, HDBSCAN
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor, KNeighborsClassifier, NearestNeighbors
from sklearn.inspection import permutation_importance
from scipy.spatial.distance import euclidean
from imblearn.over_sampling import SMOTE, BorderlineSMOTE
#from ctgan import CTGAN

### Read dataframes after preprocessing 

In [5]:
# Original df
original = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\encoded_normalized\\original_data_normalized.csv")

# KMeans + centeroid
borderline_KM_cent = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\borderline_KM_centroids.csv")
smote_KM_cent = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\smote_KM_centroids.csv")
mix_KM_cent = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\mix_KM_centroids.csv")
GAN_KM_cent = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\GAN_KM_centroids.csv")

# KMeans + the nearesrt neighbor
borderline_KM_nn = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\borderline_KM_nn.csv")
smote_KM_nn = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\smote_KM_nn.csv")
mix_KM_nn = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\mix_KM_nn.csv")
GAN_KM_nn = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\GAN_KM_nn.csv")

# KMeans + cosine similarity
borderline_KM_cos = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\borderline_KM_cos.csv")
smote_KM_cos = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\smote_KM_cos.csv")
mix_KM_cos = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\mix_KM_cos.csv")
GAN_KM_cos = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\GAN_KM_cos.csv")

# KMeans + cosine similarity + Mahalanobis distance
borderline_KM_cos_maha = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\borderline_KM_cos_mal.csv")
smote_KM_cos_maha = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\smote_KM_cos_mal.csv")
mix_KM_cos_maha = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\mix_KM_cos_mal.csv")
GAN_KM_cos_maha = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\GAN_KM_cos_mal.csv")

# HDBSCAN + the nearesrt neighbor
borderline_HDBSCAN_nn = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\borderline_HDBSCAN_NN.csv")
smote_HDBSCAN_nn = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\smote_HDBSCAN_NN.csv")
mix_HDBSCAN_nn = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\mix_HDBSCAN_NN.csv")
GAN_HDBSCAN_nn = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\GAN_HDBSCAN_NN.csv")

# HDBSCAN + cosine similarity
borderline_HDBSCAN_cos = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\borderline_HDBSCAN_cos.csv")
smote_HDBSCAN_cos = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\smote_HDBSCAN_cos.csv")
mix_HDBSCAN_cos = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\mix_HDBSCAN_cos.csv")
GAN_HDBSCAN_cos = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\GAN_HDBSCAN_cos.csv")

# Kmeans(number of samples calculated by HDBSCAN) + the nearesrt neighbor
borderline_KM_HDBSCAN_nn = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\borderline_kmeans&hdbscan_nn.csv")
smote_KM_HDBSCAN_nn = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\smote_kmeans&hdbscan_nn.csv")
mix_KM_HDBSCAN_nn = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\mix_kmeans&hdbscan_nn.csv")
GAN_KM_HDBSCAN_nn = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\GAN_kmeans&hdbscan_nn.csv")

# Kmeans(number of samples calculated by HDBSCAN) + cosine similarity
borderline_KM_HDBSCAN_cos = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\borderline_kmeans&hdbscan_cos.csv")
smote_KM_HDBSCAN_cos = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\smote_kmeans&hdbscan_cos.csv")
mix_KM_HDBSCAN_cos = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\mix_kmeans&hdbscan_cos.csv")
GAN_KM_HDBSCAN_cos = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\GAN_kmeans&hdbscan_cos.csv")  

# KMeans(number of samples calculated by HDBSCAN) + cosine similarity + Mahalanobis distance
borderline_KM_HDBSCAN_cos_maha = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\borderline_kmeans&hdbscan_cos&mal.csv")
smote_KM_HDBSCAN_cos_maha = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\smote_kmeans&hdbscan_cos&mal.csv")
mix_KM_HDBSCAN_cos_maha = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\mix_kmeans&hdbscan_cos&mal.csv")
GAN_KM_HDBSCAN_cos_maha = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\GAN_kmeans&hdbscan_cos&mal.csv")

### Machine Learning

In [ ]:
models = {}

models["DT"] = DecisionTreeClassifier()
models["DT_KM_C"] = DecisionTreeClassifier()
models["DT_KM_NN"] = DecisionTreeClassifier()
models["DT_KM_COS"] = DecisionTreeClassifier()
models["DT_KM_COS_MAHA"] = DecisionTreeClassifier()
models["DT_HDBSCAN_NN"] = DecisionTreeClassifier()
models["DT_HDBSCAN_COS"] = DecisionTreeClassifier()
models["DT_KM_SCAN_NN"] = DecisionTreeClassifier()
models["DT_KM_SCAN_COS"] = DecisionTreeClassifier()
models["DT_KM_SCAN_COS_MAHA"] = DecisionTreeClassifier()

models["RF"] = RandomForestClassifier()
models["RF_KM_C"] = RandomForestClassifier()
models["RF_KM_NN"] = RandomForestClassifier()
models["RF_KM_COS"] = RandomForestClassifier()
models["RF_KM_COS_MAHA"] = RandomForestClassifier()
models["RF_HDBSCAN_NN"] = RandomForestClassifier()
models["RF_HDBSCAN_COS"] = RandomForestClassifier()
models["RF_KM_SCAN_NN"] = RandomForestClassifier()
models["RF_KM_SCAN_COS"] = RandomForestClassifier()
models["RF_KM_SCAN_COS_MAHA"] = RandomForestClassifier()

models["XGB"] = XGBClassifier()
models["XGB_KM_C"] = XGBClassifier()
models["XGB_KM_NN"] = XGBClassifier()
models["XGB_KM_COS"] = XGBClassifier()
models["XGB_KM_COS_MAHA"] = XGBClassifier()
models["XGB_HDBSCAN_NN"] = XGBClassifier()
models["XGB_HDBSCAN_COS"] = XGBClassifier()
models["XGB_KM_SCAN_NN"] = XGBClassifier()
models["XGB_KM_SCAN_COS"] = XGBClassifier()
models["XGB_KM_SCAN_COS_MAHA"] = XGBClassifier()

models["XGBRF"] = XGBRFClassifier()
models["XGBRF_KM_C"] = XGBRFClassifier()
models["XGBRF_KM_NN"] = XGBRFClassifier()
models["XGBRF_KM_COS"] = XGBRFClassifier()
models["XGBRF_KM_COS_MAHA"] = XGBRFClassifier()
models["XGBRF_HDBSCAN_NN"] = XGBRFClassifier()
models["XGBRF_HDBSCAN_COS"] = XGBRFClassifier()
models["XGBRF_KM_SCAN_NN"] = XGBRFClassifier()
models["XGBRF_KM_SCAN_COS"] = XGBRFClassifier()
models["XGBRF_KM_SCAN_COS_MAHA"] = XGBRFClassifier()

In [11]:
data = {}

data["DT"] = (original.drop(columns=["target","source"]), original["target"])
data["DT_KM_C"] = (borderline_KM_cent.drop(columns=["target","source"]), borderline_KM_cent["target"])
data["DT_KM_NN"] = (borderline_KM_nn.drop(columns=["target","source"]), borderline_KM_nn["target"])
data["DT_KM_COS"] = (borderline_KM_cos.drop(columns=["target","source"]), borderline_KM_cos["target"])
data["DT_KM_COS_MAHA"] = (borderline_KM_cos_maha.drop(columns=["target","source"]), borderline_KM_cos_maha["target"])
data["DT_HDBSCAN_NN"] = (borderline_HDBSCAN_nn.drop(columns=["target","source"]), borderline_HDBSCAN_nn["target"])
data["DT_HDBSCAN_COS"] = (borderline_HDBSCAN_cos.drop(columns=["target","source"]), borderline_HDBSCAN_cos["target"])
data["DT_KM_SCAN_NN"] = (borderline_KM_HDBSCAN_nn.drop(columns=["target","source"]), borderline_KM_HDBSCAN_nn["target"])
data["DT_KM_SCAN_COS"] = (borderline_KM_HDBSCAN_cos.drop(columns=["target","source"]), borderline_KM_HDBSCAN_cos["target"])
data["DT_KM_SCAN_COS_MAHA"] = (borderline_KM_HDBSCAN_cos_maha.drop(columns=["target","source"]), borderline_KM_HDBSCAN_cos_maha["target"])

data["RF"] = (original.drop(columns=["target","source"]), original["target"])
data["RF_KM_C"] = (smote_KM_cent.drop(columns=["target","source"]), smote_KM_cent["target"])
data["RF_KM_NN"] = (smote_KM_nn.drop(columns=["target","source"]), smote_KM_nn["target"])
data["RF_KM_COS"] = (smote_KM_cos.drop(columns=["target","source"]), smote_KM_cos["target"])
data["RF_KM_COS_MAHA"] = (smote_KM_cos_maha.drop(columns=["target","source"]), smote_KM_cos_maha["target"])
data["RF_HDBSCAN_NN"] = (smote_HDBSCAN_nn.drop(columns=["target","source"]), smote_HDBSCAN_nn["target"])
data["RF_HDBSCAN_COS"] = (smote_HDBSCAN_cos.drop(columns=["target","source"]), smote_HDBSCAN_cos["target"])
data["RF_KM_SCAN_NN"] = (smote_KM_HDBSCAN_nn.drop(columns=["target","source"]), smote_KM_HDBSCAN_nn["target"])
data["RF_KM_SCAN_COS"] = (smote_KM_HDBSCAN_cos.drop(columns=["target","source"]), smote_KM_HDBSCAN_cos["target"])
data["RF_KM_SCAN_COS_MAHA"] = (smote_KM_HDBSCAN_cos_maha.drop(columns=["target","source"]), smote_KM_HDBSCAN_cos_maha["target"])

data["XGB"] = (original.drop(columns=["target","source"]), original["target"])
data["XGB_KM_C"] = (mix_KM_cent.drop(columns=["target","source"]), mix_KM_cent["target"])
data["XGB_KM_NN"] = (mix_KM_nn.drop(columns=["target","source"]), mix_KM_nn["target"])
data["XGB_KM_COS"] = (mix_KM_cos.drop(columns=["target","source"]), mix_KM_cos["target"])
data["XGB_KM_COS_MAHA"] = (mix_KM_cos_maha.drop(columns=["target","source"]), mix_KM_cos_maha["target"])
data["XGB_HDBSCAN_NN"] = (mix_HDBSCAN_nn.drop(columns=["target","source"]), mix_HDBSCAN_nn["target"])
data["XGB_HDBSCAN_COS"] = (mix_HDBSCAN_cos.drop(columns=["target","source"]), mix_HDBSCAN_cos["target"])
data["XGB_KM_SCAN_NN"] = (mix_KM_HDBSCAN_nn.drop(columns=["target","source"]), mix_KM_HDBSCAN_nn["target"])
data["XGB_KM_SCAN_COS"] = (mix_KM_HDBSCAN_cos.drop(columns=["target","source"]), mix_KM_HDBSCAN_cos["target"])
data["XGB_KM_SCAN_COS_MAHA"] = (mix_KM_HDBSCAN_cos_maha.drop(columns=["target","source"]), mix_KM_HDBSCAN_cos_maha["target"])

data["XGBRF"] = (original.drop(columns=["target","source"]), original["target"])
data["XGBRF_KM_C"] = (GAN_KM_cent.drop(columns=["target","source"]), GAN_KM_cent["target"])
data["XGBRF_KM_NN"] = (GAN_KM_nn.drop(columns=["target","source"]), GAN_KM_nn["target"])
data["XGBRF_KM_COS"] = (GAN_KM_cos.drop(columns=["target","source"]), GAN_KM_cos["target"])
data["XGBRF_KM_COS_MAHA"] = (GAN_KM_cos_maha.drop(columns=["target","source"]), GAN_KM_cos_maha["target"])
data["XGBRF_HDBSCAN_NN"] = (GAN_HDBSCAN_nn.drop(columns=["target","source"]), GAN_HDBSCAN_nn["target"])
data["XGBRF_HDBSCAN_COS"] = (GAN_HDBSCAN_cos.drop(columns=["target","source"]), GAN_HDBSCAN_cos["target"])
data["XGBRF_KM_SCAN_NN"] = (GAN_KM_HDBSCAN_nn.drop(columns=["target","source"]), GAN_KM_HDBSCAN_nn["target"])
data["XGBRF_KM_SCAN_COS"] = (GAN_KM_HDBSCAN_cos.drop(columns=["target","source"]), GAN_KM_HDBSCAN_cos["target"])
data["XGBRF_KM_SCAN_COS_MAHA"] = (GAN_KM_HDBSCAN_cos_maha.drop(columns=["target","source"]), GAN_KM_HDBSCAN_cos_maha["target"])

### Grid Search

In [13]:
params_ = {
    "DT": {
        'max_depth': [4, 8],
        'min_samples_leaf': [1, 2],
        'random_state': [0]
    },
    "RF": {
        'n_estimators': [40, 80],
        'max_depth': [8, 12],
        'min_samples_leaf': [1],
        'random_state': [0]
    },
    "XGB": {
        'max_depth': [8, 12],
        'learning_rate': [0.01, 0.1],
        'subsample': [0.7, 1]
    },
    "XGBRF": {
        'n_estimators': [40, 80],
        'max_depth': [8],
        'subsample': [0.7, 1],
        'random_state': [0]
    }
}

models_ = {
    "DT": DecisionTreeClassifier(),
    "RF": RandomForestClassifier(),
    "XGB": XGBClassifier(verbosity=0, use_label_encoder=False),
    "XGBRF": XGBClassifier(booster='gbtree', grow_policy='depthwise', importance_type='gain', tree_method='auto', verbosity=0, use_label_encoder=False)
}

results_ = {}

n_iter = 30

scoring_metrics = {
    'precision': 'precision',
    'accuracy': 'accuracy',
    'f1-score': 'f1',
    'roc_auc': 'roc_auc'
}


for name, (X, y) in data.items():
    print(f"\n🟦 Dataset: {name}")
    results_[name] = {}

    for model_name in models_.keys():
        print(f" 🔍 Model: {model_name}")
        model = models_[model_name]
        param_dist = params_[model_name]

        results_[name][model_name] = {}

        for metric_name, metric in scoring_metrics.items():
            print(f"    📊 Metric: {metric_name}")
            search = RandomizedSearchCV(
                estimator=model,
                param_distributions=param_dist,
                n_iter=20,
                scoring=metric,
                cv=10,
                n_jobs=-1,
                verbose=0,
                random_state=42,
                return_train_score=False
            )

            search.fit(X, y)

            results_[name][model_name][metric_name] = {
                'best_score': search.best_score_,
                'mean_test_score': np.mean(search.cv_results_['mean_test_score']),
                'best_params': search.best_params_
            }

            print(f"      ✅ Best score: {search.best_score_:.4f}")
            print(f"      📈 Mean score: {np.mean(search.cv_results_['mean_test_score']):.4f}")
            
            
summary = []

for dataset_name in results_:
    for model_name in results_[dataset_name]:
        for metric_name in results_[dataset_name][model_name]:
            entry = results_[dataset_name][model_name][metric_name]
            summary.append({
                "Dataset": dataset_name,
                "Model": model_name,
                "Metric": metric_name,
                "Best Score": entry["best_score"],
                "Mean Score": entry["mean_test_score"],
                "Best Params": entry["best_params"]
            })

df_results = pd.DataFrame(summary)



🟦 Dataset: DT
 🔍 Model: DT
    📊 Metric: precision
      ✅ Best score: 0.9833
      📈 Mean score: 0.9708
    📊 Metric: accuracy
      ✅ Best score: 0.9745
      📈 Mean score: 0.9670
    📊 Metric: f1-score
      ✅ Best score: 0.9373
      📈 Mean score: 0.9207
    📊 Metric: roc_auc


C:\Users\mateu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 4 is smaller than n_iter=20. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
C:\Users\mateu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 4 is smaller than n_iter=20. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
C:\Users\mateu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 4 is smaller than n_iter=20. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
C

      ✅ Best score: 0.9657
      📈 Mean score: 0.9587
 🔍 Model: RF
    📊 Metric: precision


C:\Users\mateu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 4 is smaller than n_iter=20. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


      ✅ Best score: 0.9300
      📈 Mean score: 0.9213
    📊 Metric: accuracy


...

      ✅ Best score: 0.9734
      📈 Mean score: 0.9726
    📊 Metric: f1-score


C:\Users\mateu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 4 is smaller than n_iter=20. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


      ✅ Best score: 0.9733
      📈 Mean score: 0.9720
    📊 Metric: roc_auc


C:\Users\mateu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 4 is smaller than n_iter=20. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


      ✅ Best score: 0.9973
      📈 Mean score: 0.9971


In [17]:
print(df_results)
df_results.to_csv("D:\\ml\\undersampling_data\\reports\\ssh\\grid_search\\ssh_results_random_search.csv", index=False)

                    Dataset  Model     Metric  Best Score  Mean Score  \
0                        DT     DT  precision    0.983333    0.970833   
1                        DT     DT   accuracy    0.974474    0.966974   
2                        DT     DT   f1-score    0.937338    0.920671   
3                        DT     DT    roc_auc    0.965667    0.958667   
4                        DT     RF  precision    0.930000    0.921250   
..                      ...    ...        ...         ...         ...   
635  XGBRF_KM_SCAN_COS_MAHA    XGB    roc_auc    0.997139    0.996542   
636  XGBRF_KM_SCAN_COS_MAHA  XGBRF  precision    0.980833    0.976422   
637  XGBRF_KM_SCAN_COS_MAHA  XGBRF   accuracy    0.973441    0.972581   
638  XGBRF_KM_SCAN_COS_MAHA  XGBRF   f1-score    0.973260    0.972034   
639  XGBRF_KM_SCAN_COS_MAHA  XGBRF    roc_auc    0.997333    0.997111   

                                           Best Params  
0    {'random_state': 0, 'min_samples_leaf': 1, 'ma...  
1    {'ra